In [1]:
import datetime as dt
import numpy as np
import pandas as pd

import sqlalchemy

from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func
from flask import Flask, jsonify, request
from datetime import timedelta

In [2]:
engine = create_engine("sqlite:///Resources/hawaii.sqlite")

# Reflect an existing database into a new model.
Base = automap_base()

# Reflect the tables.
Base.prepare(engine, reflect = True)

# Save reference to the tables.
Measurement = Base.classes.measurement
Station = Base.classes.station

# Create session
session = Session(engine)

# Setup Flask
app = Flask(__name__)

In [3]:
def determine_temps_for_date_range(start, end):
    
    # Create our session (link) from Python to the DB.
    session = Session(engine)

    # If we have both a start date and an end date.
    if end != None:
        temperature_data = session.query(func.min(Measurement.tobs),\
                                         func.avg(Measurement.tobs),\
                                         func.max(Measurement.tobs)).\
                                         filter(Measurement.date >= start).\
                                         filter(Measurement.date <= end).all()
    # If we only have a start date.
    else:
        temperature_data = session.query(func.min(Measurement.tobs),\
                                         func.avg(Measurement.tobs),\
                                         func.max(Measurement.tobs)).\
                                         filter(Measurement.date >= start).all()

    session.close()

    # Convert the query results to a list.
    temperature_list = []
    no_temperature_data = False
    for min_temp, avg_temp, max_temp in temperature_data:
        if min_temp == None or avg_temp == None or max_temp == None:
            no_temperature_data = True
        temperature_list.append(min_temp)
        temperature_list.append(avg_temp)
        temperature_list.append(max_temp)
    # Return the JSON representation of dictionary.
    if no_temperature_data == True:
        return f"No temperature data found for the given date range. Try another date range."
    else:
        return print(temperature_list)

In [10]:
start = "2016-12-25"
end = "2017-01-03"
determine_temps_for_date_range(start, end)

[62.0, 70.81666666666666, 78.0]


In [5]:
@app.route("/api/v1.0/<start_date>")
def temp_start(start_date):

    session = Session(engine)

    temperature_data = session.query(func.min(Measurement.tobs),\
                                func.avg(Measurement.tobs),\
                                func.max(Measurement.tobs)).\
                                filter(Measurement.date >= start_date).all()

    session.close()

    temp_all = []
    for min, avg, max in temperature_data:
        temp_dict = {}
        temp_dict["Min"] = min
        temp_dict["Average"] = avg
        temp_dict["Max"] = max
        temp_all.append(temp_dict)

    return print(temp_all)

@app.route("/api/v1.0/<start_date>/<end_date>")
def temp_start_end(start_date, end_date):
    
    session = Session(engine)
    
    temperature_data = session.query(func.min(Measurement.tobs),\
                                func.avg(Measurement.tobs),\
                                func.max(Measurement.tobs)).\
                                filter(Measurement.date >= start_date).\
                                filter(Measurement.date <= end_date).all()
    session.close()

    temp_all = []
    for min,avg,max in temperature_data:
        temp_dict = {}
        temp_dict["Min"] = min
        temp_dict["Average"] = avg
        temp_dict["Max"] = max
        temp_all.append(temp_dict)

    return print(temp_all)

In [6]:
start_date = "2016-12-25"
end_date = "2017-01-03"
temp_start_end(start_date, end_date)

[{'Min': 62.0, 'Average': 70.81666666666666, 'Max': 78.0}]


In [7]:
temp_start(start_date)

[{'Min': 58.0, 'Average': 74.06944444444444, 'Max': 87.0}]
